 #  A Whale off the Port(folio)

 In this assignment, you'll get to use what you've learned this week to evaluate the performance among various algorithmic, hedge, and mutual fund portfolios and compare them against the S&P 500.

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
%matplotlib inline

# Data Cleaning

In this section, you will need to read the CSV files into DataFrames and perform any necessary data cleaning steps. After cleaning, combine all DataFrames into a single DataFrame.

Files:
1. whale_returns.csv
2. algo_returns.csv
3. sp500_history.csv

## Whale Returns

Read the Whale Portfolio daily returns and clean the data

In [ ]:
# Reading whale returns
whale_returns_csv = Path("Resources/whale_returns.csv")
# YOUR CODE HERE
whale_df = pd.read_csv(whale_returns_csv, index_col="Date", parse_dates=True)
whale_df.head()

In [ ]:
# Count nulls
# YOUR CODE HERE
whale_df.isnull().sum()

In [ ]:
# Drop nulls
# YOUR CODE HERE
whale_df.dropna(inplace=True)
whale_df.isnull().sum()
# whale_df.head() to double check if dropped first row

## Algorithmic Daily Returns

Read the algorithmic daily returns and clean the data

In [ ]:
# Reading algorithmic returns 
algo_returns_csv = Path("Resources/algo_returns.csv")
# YOUR CODE HERE
algo_df = pd.read_csv(algo_returns_csv,index_col="Date", parse_dates=True)

In [ ]:
# Count nulls
# YOUR CODE HERE
algo_df.isnull().sum()

In [ ]:
# Drop nulls
# YOUR CODE HERE
algo_df.dropna(inplace=True)
algo_df.isnull().sum()

## S&P 500 Returns

Read the S&P500 Historic Closing Prices and create a new daily returns DataFrame from the data. 

In [ ]:
# Reading S&P 500 Closing Prices
sp500_history_csv = Path("Resources/sp500_history.csv")
# YOUR CODE HERE
sp500_df = pd.read_csv(sp500_history_csv, index_col="Date", parse_dates=True)
sp500_df = sp500_df.sort_index(ascending=True)
sp500_df.head()

In [ ]:
# Check Data Types
# YOUR CODE HERE
sp500_df.dtypes

In [ ]:
#fix data type and get rid of $
sp500_df["Close"] = sp500_df["Close"].str.replace('$','')
sp500_df.head()

In [ ]:
sp500_df.dtypes
sp500_df["Close"] = sp500_df["Close"].astype(float)
sp500_df.dtypes

In [ ]:
# Calculate Daily Returns
# YOUR CODE HERE

sp_daily = sp500_df.pct_change()
sp_daily

In [ ]:
# Drop nulls
# YOUR CODE HERE
sp_daily.isnull().sum()
sp_daily.dropna(inplace=True)
sp_daily.isnull().sum()

In [ ]:
# Rename Column
# YOUR CODE HERE

sp_daily.columns = ["SP 500"]

sp_daily

## Combine Whale, Algorithmic, and S&P 500 Returns

In [ ]:
# Concatenate all DataFrames into a single DataFrame
# YOUR CODE HERE

big_portfolio = pd.concat([algo_df, sp_daily, whale_df], axis="columns", join= "inner")
big_portfolio

---

# Portfolio Analysis

In this section, you will calculate and visualize performance and risk metrics for the portfolios.

## Performance

Calculate and Plot the daily returns and cumulative returns. Does any portfolio outperform the S&P 500? 

In [ ]:
# Plot daily returns
# YOUR CODE HERE
daily_return_plot = big_portfolio.plot(figsize=(20,10))
daily_return_plot

In [ ]:
# Plot cumulative returns
# YOUR CODE HERE
cum_returns = (1 + big_portfolio).cumprod()-1
cum_returns.plot(figsize=(20,10))

## Risk

Determine the _risk_ of each portfolio:

1. Create a box plot for each portfolio. 
2. Calculate the standard deviation for all portfolios
4. Determine which portfolios are riskier than the S&P 500
5. Calculate the Annualized Standard Deviation

In [ ]:
# Box plot to visually show risk
# YOUR CODE HERE
big_portfolio.boxplot(grid=False, rot=100000000, fontsize=10)

In [ ]:
# Daily Standard Deviations
# Calculate the standard deviation for each portfolio. 
# Which portfolios are riskier than the S&P 500? 
# The portfolios with higher standard deviations are the most risky
# YOUR CODE HERE
daily_std_dev = big_portfolio.std()
daily_std_dev

In [ ]:
# ranking of least risky std deviations... #1 being least risky
daily_std_dev.rank(method='min')

In [ ]:
# Determine which portfolios are riskier than the S&P 500
# YOUR CODE HERE
greater_than_SP500 = daily_std_dev[daily_std_dev>daily_std_dev["SP 500"]]
print(greater_than_SP500)

In [ ]:
# Calculate the annualized standard deviation (252 trading days)
# YOUR CODE HERE
annual_std_dev = big_portfolio.std() * np.sqrt(252)
annual_std_dev

---

## Rolling Statistics

Risk changes over time. Analyze the rolling statistics for Risk and Beta. 

1. Plot the rolling standard deviation of the various portfolios along with the rolling standard deviation of the S&P 500 (consider a 21 day window). Does the risk increase for each of the portfolios at the same time risk increases in the S&P?
2. Construct a correlation table for the algorithmic, whale, and S&P 500 returns. Which returns most closely mimic the S&P?
3. Choose one portfolio and plot a rolling beta between that portfolio's returns and S&P 500 returns. Does the portfolio seem sensitive to movements in the S&P 500?
4. An alternative way to calculate a rolling window is to take the exponentially weighted moving average. This is like a moving window average, but it assigns greater importance to more recent observations. Try calculating the ewm with a 21 day half-life.

In [ ]:
# Calculate and plot the rolling standard deviation for
# the S&P 500 and whale portfolios using a 21 trading day window
# YOUR CODE HERE
big_portfolio.rolling(window=21).std().plot()

In [ ]:
# Construct a correlation table
# YOUR CODE HERE
correlation_table = big_portfolio.corr()
correlation_table

In [ ]:
# Calculate Beta for a single portfolio compared to the total market (S&P 500)
# (Your graph may differ, dependent upon which portfolio you are comparing)
# YOUR CODE HERE
soros_cov = big_portfolio["SOROS FUND MANAGEMENT LLC"].cov(big_portfolio["SP 500"])

soros_var = big_portfolio["SOROS FUND MANAGEMENT LLC"].var()

soros_beta = soros_cov/ soros_var
soros_beta

In [ ]:
soros_rolling_covariance = big_portfolio["SOROS FUND MANAGEMENT LLC"].rolling(window=30).cov(big_portfolio["SP 500"])
soros_rolling_variance = big_portfolio["SP 500"].rolling(window=30).var()

soros_rolling_beta = soros_rolling_covariance / soros_rolling_variance

soros_rolling_beta.plot()

In [ ]:
# Calculate a rolling window using the exponentially weighted moving average. 
# YOUR CODE HERE

---

## Sharpe Ratios
In reality, investment managers and thier institutional investors look at the ratio of return-to-risk, and not just returns alone. (After all, if you could invest in one of two portfolios, each offered the same 10% return, yet one offered lower risk, you'd take that one, right?)

1. Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot.
2. Determine whether the algorithmic strategies outperform both the market (S&P 500) and the whales portfolios.

In [ ]:
# Calculate annualized Sharpe Ratios
# YOUR CODE HERE
sharpe = big_portfolio.mean() / big_portfolio.std() * np.sqrt(252)
sharpe

In [ ]:
# Visualize the sharpe ratios as a bar plot
# YOUR CODE HERE
sharpe.plot.bar()

On the basis of this performance metric, do our algo strategies outperform both 'the market' and the whales? Type your answer here: Algo 1 does for sure. Algo 2 is better than every whale except for Berk, and worse than SP500

---

# Portfolio Returns

In this section, you will build your own portfolio of stocks, calculate the returns, and compare the results to the Whale Portfolios and the S&P 500. 

1. Visit [Google Sheets](https://docs.google.com/spreadsheets/) and use the in-built Google Finance function to choose 3-5 stocks for your own portfolio.
2. Download the data as CSV files and calculate the portfolio returns.
3. Calculate the returns for each stock.
4. Using those returns, calculate the weighted returns for your entire portfolio assuming an equal number of shares for each stock.
5. Add your portfolio returns to the DataFrame with the other portfolios and rerun the analysis. How does your portfolio fair?


## Your analysis should include the following:

- Using all portfolios:
 - The annualized standard deviation (252 trading days) for all portfolios.
 - The plotted rolling standard deviation using a 21 trading day window for all portfolios.
 - The calculated annualized Sharpe Ratios and the accompanying bar plot visualization.
 - A correlation table.
- Using your custom portfolio and one other of your choosing:
 - The plotted beta. 

In [ ]:
# Read the first stock
# YOUR CODE HERE

aapl_historical = ("Resources/aapl_historical.csv")
aapl_df = pd.read_csv(aapl_historical, index_col="Trade DATE", parse_dates=True)
aapl_df = aapl_df.sort_index()
aapl_df

In [ ]:
# Read the second stock
# YOUR CODE HERE
cost_historical = ("Resources/cost_historical.csv")
cost_df = pd.read_csv(cost_historical, index_col="Trade DATE", parse_dates=True)
cost_df = cost_df.sort_index()
cost_df.head()


In [ ]:
# Read the third stock
# YOUR CODE HERE

goog_historical = ("Resources/goog_historical.csv")
goog_df = pd.read_csv(goog_historical, index_col="Trade DATE", parse_dates=True)
goog_df = goog_df.sort_index()
goog_df.head()

In [ ]:
# Concatenate all stocks into a single DataFrame
# YOUR CODE HERE
my_portfolio = pd.concat([goog_df, aapl_df, cost_df], axis="rows",join="inner")

my_portfolio

In [ ]:
# Reset the index
# YOUR CODE HERE
# not sure why i need to reset index, so i didn't
my_portfolio=my_portfolio.reset_index()
my_portfolio

In [ ]:
# Pivot so that each column of prices represents a unique symbol
# YOUR CODE HERE
Overstock=my_portfolio.pivot_table(values="NOCP",index="Trade DATE",columns="Symbol")
Overstock

In [ ]:
# Drop Nulls
# YOUR CODE HERE
my_portfolio.isnull().sum()

## Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock

In [ ]:
# Calculate weighted portfolio returns
weights = [1/3, 1/3, 1/3]
# YOUR CODE HERE
my_returns = my_portfolio.pct_change()
my_returns.head()
my_returns.dropna()

## Join your portfolio returns to the DataFrame that contains all of the portfolio returns

In [ ]:
# Add your "Custom" portfolio to the larger dataframe of fund returns
# YOUR CODE HERE
megafolio = pd.concat([my_returns, big_portfolio], axis="columns", join="inner")
megafolio.dropna()

columns = ["goog", "aapl", "cost","Algo 1", "Algo 2", "SP 500", "SOROS FUND MANAGEMENT LLC", "PAULSON & CO.INC.", "TIGER GLOBAL MANAGEMENT LLC", "BERKSHIRE HATHAWAY INC"]
megafolio.columns = columns
megafolio

## Re-run the performance and risk analysis with your portfolio to see how it compares to the others

In [ ]:
# Risk
# YOUR CODE HERE
risk = megafolio.std()
risk

In [ ]:
# Rolling
# YOUR CODE HERE
megafolio.rolling(window=21).std().plot()

In [ ]:
# Annualized Sharpe Ratios
# YOUR CODE HERE
sharpe1 = megafolio.mean() / megafolio.std() * np.sqrt(252)
sharpe1

In [ ]:
# Visualize the sharpe ratios as a bar plot
# YOUR CODE HERE
sharpe1.plot.bar()

In [ ]:
# Create a correlation analysis
# YOUR CODE HERE
correlation_table1 = megafolio.corr()
correlation_table1

In [ ]:
# Beta
# YOUR CODE HERE
mega_cov = megafolio.cov()

mega_var = megafolio.var()

mega_beta = mega_cov/ mega_var
mega_beta